In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
# !pip install spacy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

stopwords=list(STOP_WORDS)
from string import punctuation
# !python -m spacy download en_core_web_sm

nlp_spacy = spacy.load('en_core_web_sm')
import nltk

In [3]:
from nltk.corpus import wordnet

In [4]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [18]:
class text_summary:
    def __init__(self, model='nltk'):
        self.model = model
#         from string import punctuation
#         if self.model =='nltk':
#             import nltk
#             nltk.download('punkt')
#         elif self.model =='spacy':
#             import spacy
#             from spacy.lang.en.stop_words import STOP_WORDS
#             stopwords=list(STOP_WORDS)
#             nlp_spacy = spacy.load('en_core_web_sm')
#         else:
#             pass

    
    def clean_words(self, text, remove_stop_words = True):
        """it removes the punctuation and numbers from the text, optional: stop words"""
        if remove_stop_words:
            text = text.split()
#             print(text)
            text = " ".join([x for x in text if x not in stopwords])
        text  = "".join([char for char in text if char not in punctuation])
        text = re.sub('[0-9]+', '', text)   
        text = text.lower().replace('\n', ' ')
        text = re.sub("\s+", " ", text)
#         print(text)
        return text
    
    def clean_data_text(self, df):
        self.data = df
    # also included removing stop words part but not in this case later we will check
        self.data['tokenized_sents'] = self.data['tokenized_sents'].apply(self.clean_words)
    
        return self.data
    

    def create_sent(self, df):

    #     self.data['sentences_list'] = self.data['article_text'].apply(nltk.sent_tokenize)
        self.data = df
        self.data['article_text'] =self.data['article_text'].astype('str')
        if self.model =='spacy':
            self.data['nlp_docs'] = self.data['article_text'].apply(nlp_spacy)
            # iterating over all the every doc to get the sentecnces
            self.data['sentence_list'] = ''
            for i in range(self.data.shape[0]):
                sentence=  [str(sent) for sent in self.data['nlp_docs'][i].sents]
                self.data.at[self.data.index[i],'sentence_list'] = sentence

        elif self.model == 'nltk':
            self.data['sentence_list'] = self.data['article_text'].apply(nltk.sent_tokenize)
        else:
            pass

        self.data['tokenized_sents'] = self.data['article_text'].astype(str)
        self.data = self.clean_data_text(self.data) # cleaning the article
        self.data['tokenized_sents_copy'] = self.data['tokenized_sents'] #creating copy of sentence without changes
        return self.data
    
    def create_words_frequency_dictionary(self, df):
        self.data = df
        from collections import Counter
        self.data['frequency'] = self.data['tokenized_sents_copy'].str.split(' ').apply(Counter)
        return self.data
    
    def get_maximum_frequency(self, freq_dic):
        return max(freq_dic.values())
    
    def create_word_weighted_freq(self, row_dictionary):
        max_weight = self.get_maximum_frequency(row_dictionary)
        for key, val in row_dictionary.items():
            row_dictionary[key] = val/max_weight
        return row_dictionary
    
#     return max key value pair
#     this is not needed but still created it for future use
    def get_max_key_value(self, freq_dic):
        max_key_value_dic = {}
        k = max(freq_dic, key = freq_dic.get)
        max_key_value_dic[k] = freq_dic[k]
        return max_key_value_dic
    
#     test_function
    def insert_max_word_key_value(self, df):
        self.data = df
        self.data['max'] = self.data['frequency'].apply(self.get_max_key_value)
        return self.data
    
    def calculate_sentence_score(self, sentence_list, 
                             word_weight_dictionary,
                            decided_len=30, short_sent_length = 5):
        """This function calculate the sentence score of a sentence by 
        summing up the weighted score of the words. """
        sentence_score = {}
    #     print('step 1')
        for sent in sentence_list:
            sent_len = len(sent.split(' '))
            if ((sent_len < decided_len) and (sent_len > short_sent_length)):
                for word in sent.lower():
                    if sent not in sentence_score.keys():
                        sentence_score[sent] = word_weight_dictionary[word]
                    else:
                        sentence_score[sent] += word_weight_dictionary[word]
  
        return [sentence_score]

    def remove_ascii(self, text):
        text = str(text)
        text = ''.join((c for c in text if ord(c) < 128))
        return text
            
    
    
    def topic_tagging(self, topic_dict, text):
        """This function tag the articles with topics using
        the given keywords dictionary.
        """
        tagg_value_dic = {}
        # initialising a dictionary with keys only
        for k in topic_dict.keys():
            tagg_value_dic[k] = 0
        clean_text = self.clean_words(text)
        clean_text = clean_text.split()

        for k, v in topic_dict.items():
            for word in clean_text:
                if word in v:
                    tagg_value_dic[k] += 1
                
        return tagg_value_dic
    
    
    def create_synonms_from_keywords(self, kwords):
        """This function creates a dictionary of synonyms 
        of words which can be used in getting the summary."""
        synonyms_dict={}
        
        keywords = kwords
        
        for word1 in keywords:
            synonyms_dict[word1]={}
            synonyms=[]
            for syn in wordnet.synsets(word1):
                #print("Yes")
                for l in syn.lemmas():
                    synonyms.append(l.name())
            synonyms_dict[word1]=(synonyms)
        print(synonyms_dict)
        return synonyms_dict
    
    def insert_topics_of_article(self, data, topic_dict):
        self.df = data
        for i in range(self.df.shape[0]):
            self.df.at[i, 'topics'] = self.topic_tagging(topic_dict,
                                                         self.df.at[i,'article_text'])
        
        return self.df
    
    


    
    def create_summary(self, data, 
                       keywords = [],
                       summary_sent_len =7):
        """It calculates the summary on the basis of the highest sentence score.
        It also has a optional keywords parameter which can be used to pass a list 
        of words, if present in any sentence then it will be part of the summary, no matter
        what is the sentence summary."""
        self.df = data
        self.df['sentences_scores_dict'] = 0
        self.df['summary'] = ''
        
        self.df['summary'] = self.df['summary'].astype('O')
        for i in range(self.df.shape[0]):
            dictionary_of_sent_scores = self.calculate_sentence_score(self.df['sentence_list'][i],
                                                                 word_weight_dictionary=self.df['weighted_freq'][i]) 

            

            dictionary_of_sent_scores = dictionary_of_sent_scores[0]
            sorted_dict = sorted(dictionary_of_sent_scores,
                                         key = dictionary_of_sent_scores.get,
                                reverse=True)
            
#             if keywords are passed then adding them by increasing the score
#             of those sentences which contains these keywords and also increasing
#             the variable length so that it doesn't skip important sentences
#             on the basis of weights.
            keywords_sentences_list = [] 
            if len(keywords) > 0:
                
                for k in dictionary_of_sent_scores.keys(): 
                    if any(word in k for word in keywords):
                        keywords_sentences_list.append(k)
    
            
            keywords_sentences_list = keywords_sentences_list + sorted_dict
        
            self.df.loc[i, 'summary'] = " ".join(keywords_sentences_list[:summary_sent_len])
            
#             including tagging here
#             self.df.loc[i,'topic_tagging'] = 
            
        return self.df
    

In [6]:
y = 'this is my, . - ! ciytâ€œ'

In [7]:
text = ''.join((c for c in y if ord(c) < 128))
print(text)

this is my, . - ! ciyt


In [22]:
import pandas as pd
if __name__ == "__main__":
#     loading data
#     df = pd.read_csv('tennis_articles.csv',encoding = 'unicode_escape')
#     creating copy of data
    
    df = pd.read_excel('COPS Automation Web Scrape\Master_consolidated3.xlsx')
    
    df.rename(columns={'Brief Description':'article_text',
                        'Headlines':'article_title',
                        'Link':'source',
                        }, inplace=True)
    
    df_nltk = df.copy()
    df_spacy = df.copy()
#     creating instances of class
    spacy_nlp_model = text_summary(model='spacy')
    nltk_nlp_model = text_summary(model='nltk')
    
#     Remove non ascii characters
    df_spacy['article_text'] = df_spacy['article_text'].apply(spacy_nlp_model.remove_ascii)
    df_nltk['article_text'] = df_nltk['article_text'].apply(nltk_nlp_model.remove_ascii)
    
    

    
#     creating sentences
    df_spacy = spacy_nlp_model.create_sent(df_spacy)
    df_nltk = nltk_nlp_model.create_sent(df_nltk)
#     creating word dictionaries
    df_spacy = spacy_nlp_model.create_words_frequency_dictionary(df_spacy)
    df_nltk = nltk_nlp_model.create_words_frequency_dictionary(df_nltk)
#     It is a test function
    df_spacy = spacy_nlp_model.insert_max_word_key_value(df_spacy)
    df_nltk = nltk_nlp_model.insert_max_word_key_value(df_nltk)

    
# this function has one bug, it is changing the main column of frequency also.
    df_spacy['weighted_freq'] = df_spacy['frequency'].apply(spacy_nlp_model.create_word_weighted_freq)
    df_nltk['weighted_freq'] = df_nltk['frequency'].apply(nltk_nlp_model.create_word_weighted_freq)
# loading key words    
    k = pd.read_excel('keyword.xlsx')

    kwords = list(k['Keywords'])
    
#     Article tagging
#     spacy_kwords_dict = spacy_nlp_model.create_synonms_from_keywords(kwords)
#     nltk_kwords_dict = nltk_nlp_model.create_synonms_from_keywords(kwords)
    
#     df_spacy = spacy_nlp_model.insert_topics_of_article(df_spacy, spacy_kwords_dict)
#     df_nltk = nltk_nlp_model.insert_topics_of_article(df_nltk, nltk_kwords_dict)
    
    
    
    
#     for spacy first  
    df_spacy = spacy_nlp_model.create_summary(df_spacy, keywords= kwords)
#     for nltk 
    df_nltk = nltk_nlp_model.create_summary(df_nltk)
#     saving the ouput in csv file


#     removing extra columns before writing
    df_spacy.drop(columns=['nlp_docs', 'sentence_list', 'tokenized_sents',
       'tokenized_sents_copy', 'frequency', 'max', 'weighted_freq',
       'sentences_scores_dict']).to_csv('spacy_summary.csv')
    df_nltk.drop(columns=['sentence_list', 'tokenized_sents',
       'tokenized_sents_copy', 'frequency', 'max', 'weighted_freq',
       'sentences_scores_dict']).to_csv('nltk_summary.csv')
    
    # add one more thing. summary sentences should be in same orders as in 
    # articles.
    

